# Basic inliner example

This notebook walks through a simple inlining example to showcase the essential mechanisms of the `Inline` class.

In [1]:
from inliner import Inliner
import inspect

Let's say we have a basic API like in `basic_api.py`. The source code for the function `foo` is shown below.

In [2]:
from basic_api import foo
print(inspect.getsource(foo))

def foo(x, flag=False):
    if flag:
        return x
    else:
        return x + 1



We can use the API as follows:

In [3]:
def test_api():
    from basic_api import foo
    assert foo(1) == 2
    assert foo(1, flag=True) == 1

The goal of the inliner is to unwrap all code from the `basic_api` module, here just the `foo` function. Calling `Inliner.inline` will take all the `basic_api` functions in the code, and paste in their source.

In [4]:
inliner = Inliner(test_api, ['basic_api'])
inliner.inline()
print(inliner.make_program(comments=True))

from basic_api import foo

# foo(1)
x___foo = 1
flag___foo = False
if flag___foo:
    if 'var0' not in locals() and 'var0' not in globals():
        var0 = x___foo
elif 'var0' not in locals() and 'var0' not in globals():
    var0 = x___foo + 1
if 'var0' not in locals() and 'var0' not in globals():
    if 'var0' not in locals() and 'var0' not in globals():
        var0 = None
assert var0 == 2

# foo(1, flag=True)
x___foo = 1
flag___foo = True
if flag___foo:
    if 'var1' not in locals() and 'var1' not in globals():
        var1 = x___foo
elif 'var1' not in locals() and 'var1' not in globals():
    var1 = x___foo + 1
if 'var1' not in locals() and 'var1' not in globals():
    if 'var1' not in locals() and 'var1' not in globals():
        var1 = None
assert var1 == 1


This code isn't very readable. The inlined function has a number of ugly checks to proxy behavior like early returns. However, we can perform a runtime analysis of this code to see which are executed, and remove them. `Inliner.deadcode` will do this analysis, and `Inliner.fixpoint` will run `deadcode` until it produces no changes.

In [5]:
inliner.fixpoint(inliner.deadcode)
print(inliner.make_program(comments=True))

from basic_api import foo

# foo(1)
x___foo = 1
flag___foo = False
var0 = x___foo + 1
assert var0 == 2

# foo(1, flag=True)
x___foo = 1
flag___foo = True
var1 = x___foo
assert var1 == 1


After simplifying the code, we have removed all of the `if` statements. However, now we have unread variables, since `flag___foo` is no longer used. We can use `Inliner.unread_vars` to find and eliminate these variables.

In [6]:
inliner.unread_vars()
print(inliner.make_program(comments=True))


# foo(1)
x___foo = 1
var0 = x___foo + 1
assert var0 == 2

# foo(1, flag=True)
x___foo = 1
var1 = x___foo
assert var1 == 1


Finally, we can make the program prettier by removing the variable suffixes used to distinguish variables in the intermediate steps.

In [7]:
inliner.remove_suffixes()
print(inliner.make_program(comments=True))


# foo(1)
x = 1
var0 = x + 1
assert var0 == 2

# foo(1, flag=True)
x = 1
var1 = x
assert var1 == 1


We can now directly execute this program to confirm that it works as the original does.

In [8]:
globls = {}
exec(inliner.make_program(), globls, globls)